https://www.hani.co.kr/arti/culture/entertainment/816895.html

In [2]:
!pip install --upgrade pip
#!pip install gensim
!pip3 install newspaper3k
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 5.0 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=e998b0b1da90f92c04d0c0eb420b969b9c5914542f66db2070f9487cce35c05b
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfi

In [4]:
from newspaper import Article
import re

def crawling(url):
    # url = input('url을 입력하세요')
    article = Article(url,laguage='ko')
    article.download()
    article.parse()
    title = article.title
    text = article.text
    text = '.'.join(article.text.split('.')[:-2:])
    #text = ''.join(text.split('기자) ')[1]) # split 할 경우
    # text = re.search(r'기자\)\s*(.*)', text)[1] # 정규표현식으로 할경우
    return title , text

import pandas as pd
url = input("검사를 위한 URL을 입력해주세요: ")
title, text = crawling(url)
data = {'text': [text]}
df = pd.DataFrame(data)

from kiwipiepy import Kiwi
kiwi = Kiwi()
def extract_nouns(text):
    result = kiwi.tokenize(text)
    for token in result:
        if token.tag in ['NNG', 'NNP']:
            yield token.form

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=extract_nouns, min_df=1)
dtm = cv.fit_transform(df.text)

word_count = pd.DataFrame({
    'word': cv.get_feature_names_out(),
    'count': dtm.sum(axis=0).flat
})


top5_keywords = word_count.sort_values('count', ascending=False).head(5)
for i in  range(5):
  word, count = top5_keywords.iloc[i]
  print("{}의 빈도수는 {}".format(word, count))

검사를 위한 URL을 입력해주세요:  https://www.hani.co.kr/arti/culture/entertainment/816895.html


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


결혼식의 빈도수는 4
배우의 빈도수는 3
송중기의 빈도수는 3
후예의 빈도수는 2
태양의 빈도수는 2
